In [ ]:
# 02_Case_Representation.ipynb

import pandas as pd
import re
import os
import json # Untuk menyimpan ke format JSON
import nltk
from nltk.tokenize import word_tokenize
from google.colab import drive

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# --- Konfigurasi Path (SESUAIKAN DENGAN skrip untitled1.py Anda) ---
# Ambil variabel ini dari skrip Tahap 1 Anda (untitled1.py)
BASE_DRIVE_PATH = "/content/drive/MyDrive/Semester 6/PK/UAS" # Sesuaikan jika berbeda dari untitled1.py
KEYWORD_FOR_FILENAMING = "perceraian" # Sesuaikan dengan yang Anda gunakan di untitled1.py

PATH_RAW_TEXT_OUTPUT = os.path.join(BASE_DRIVE_PATH, "data/raw")
PATH_INITIAL_SCRAPER_CSV_DIR = os.path.join(BASE_DRIVE_PATH, "Scrap_CSVs") # Direktori tempat CSV Tahap 1 disimpan
PATH_PROCESSED_DATA_OUTPUT = os.path.join(BASE_DRIVE_PATH, "data/processed") # Tempat output Tahap 2

# Membuat direktori output jika belum ada
os.makedirs(PATH_PROCESSED_DATA_OUTPUT, exist_ok=True)

In [ ]:
# --- 1. Muat Data Metadata dari CSV Hasil Tahap 1 ---
# Cari file CSV terbaru berdasarkan KEYWORD_FOR_FILENAMING di PATH_INITIAL_SCRAPER_CSV_DIR
try:
    csv_files = [f for f in os.listdir(PATH_INITIAL_SCRAPER_CSV_DIR) if f.startswith(f"putusan_ma_{KEYWORD_FOR_FILENAMING.replace(' ', '_')}") and f.endswith(".csv")]
    if not csv_files:
        raise FileNotFoundError(f"Tidak ada file CSV yang cocok dengan keyword '{KEYWORD_FOR_FILENAMING}' di '{PATH_INITIAL_SCRAPER_CSV_DIR}'")
    # Urutkan berdasarkan tanggal modifikasi untuk mendapatkan yang terbaru (atau berdasarkan nama jika tanggal ada di nama)
    csv_files.sort(key=lambda name: os.path.getmtime(os.path.join(PATH_INITIAL_SCRAPER_CSV_DIR, name)), reverse=True)
    LATEST_SCRAPER_CSV = os.path.join(PATH_INITIAL_SCRAPER_CSV_DIR, csv_files[0])
    print(f"Memuat metadata dari file CSV Tahap 1 terbaru: {LATEST_SCRAPER_CSV}")
    df_metadata = pd.read_csv(LATEST_SCRAPER_CSV)
except FileNotFoundError as e:
    print(f"Error: {e}")
    print("Pastikan Tahap 1 sudah dijalankan dan menghasilkan file CSV di direktori yang benar.")
    exit()
except Exception as e:
    print(f"Terjadi error saat mencoba menemukan atau memuat file CSV: {e}")
    exit()

print(f"Metadata awal dimuat. Jumlah baris: {len(df_metadata)}, Kolom: {df_metadata.columns.tolist()}")

# Verifikasi kolom penting dari CSV Tahap 1
required_cols_t1 = ['nama_file_raw_text', 'nomor_perkara', 'tanggal_putusan', 'jenis_perkara', 'amar_putusan', 'case_id']
missing_cols = [col for col in required_cols_t1 if col not in df_metadata.columns]
if missing_cols:
    print(f"ERROR: Kolom berikut tidak ditemukan di CSV Tahap 1: {missing_cols}. Kolom ini diperlukan.")
    print(f"Kolom yang ada: {df_metadata.columns.tolist()}")
    exit()

Memuat metadata dari file CSV Tahap 1 terbaru: /content/drive/MyDrive/Semester 6/PK/UAS/Scrap_CSVs/putusan_ma_perceraian_2025-06-05.csv
Metadata awal dimuat. Jumlah baris: 29, Kolom: ['case_id', 'judul_putusan', 'nomor_perkara', 'tanggal_putusan', 'jenis_perkara', 'pasal_digunakan', 'nama_pihak', 'tingkat_proses', 'kata_kunci', 'tahun_dokumen', 'tanggal_register', 'lembaga_peradilan', 'amar_putusan', 'link_sumber', 'link_pdf', 'nama_file_pdf', 'nama_file_raw_text', 'full_text_putusan_preview']


In [ ]:
# --- 2. Fungsi untuk Memuat Teks Bersih dari File .txt ---
def load_cleaned_text_from_file(filename, raw_text_path_dir):
    """Memuat teks bersih dari file .txt berdasarkan nama file."""
    if pd.isna(filename) or filename == "" or filename == "N/A":
        return "" # Kembalikan string kosong jika nama file tidak valid

    filepath = os.path.join(raw_text_path_dir, str(filename))
    try:
        if os.path.exists(filepath):
            with open(filepath, 'r', encoding='utf-8') as f:
                return f.read()
        else:
            # print(f"File teks tidak ditemukan: {filepath}") # Aktifkan untuk debugging
            return "" # Kembalikan string kosong jika file tidak ada
    except Exception as e:
        # print(f"Error saat memuat file {filepath}: {e}") # Aktifkan untuk debugging
        return ""

print(f"\nMemuat teks lengkap dari direktori: {PATH_RAW_TEXT_OUTPUT}...")
df_metadata['text_full_cleaned'] = df_metadata['nama_file_raw_text'].apply(
    lambda x: load_cleaned_text_from_file(x, PATH_RAW_TEXT_OUTPUT)
)

# Filter baris yang teksnya tidak berhasil dimuat (jika 'text_full_cleaned' kosong)
original_count = len(df_metadata)
df_metadata = df_metadata[df_metadata['text_full_cleaned'] != ""]
print(f"Jumlah baris setelah memfilter yang gagal memuat teks: {len(df_metadata)} (dari {original_count} baris awal)")

if df_metadata.empty:
    print("Tidak ada data kasus yang memiliki teks lengkap. Proses Tahap 2 tidak dapat dilanjutkan.")
    exit()


Memuat teks lengkap dari direktori: /content/drive/MyDrive/Semester 6/PK/UAS/data/raw...
Jumlah baris setelah memfilter yang gagal memuat teks: 18 (dari 29 baris awal)


In [ ]:
# --- 3. Ekstraksi Metadata Tambahan & Konten Kunci dari Teks ---
# Fungsi-fungsi ekstraksi ini bersifat heuristik dan mungkin perlu penyesuaian.

def extract_pihak_from_text(text, nomor_perkara_text):
    """Mencoba mengekstrak pihak dari teks awal putusan."""
    # Bersihkan nomor perkara di awal teks jika ada
    if pd.notna(nomor_perkara_text) and nomor_perkara_text != "":
         text_no_header = re.sub(r"^\s*(?:PUTUSAN|PENETAPAN)\s*\n?\s*No(?:mor)?\s*[:.]?\s*" + re.escape(str(nomor_perkara_text)) + r"\s*\n?", "", text, flags=re.IGNORECASE | re.MULTILINE, count=1).strip()
    else:
         text_no_header = text

    # Pola dasar untuk Penggugat/Pemohon
    penggugat_str = "N/A"
    # Mencari blok teks setelah "antara" hingga sebelum "melawan" atau "terhadap" atau penanda akhir blok penggugat
    match_penggugat_block = re.search(r"(?:antara|antara lain)\s*[:\n]([\s\S]+?)(?:;\s*\n?\s*(?:melawan|terhadap)|;\s*dan\s*untuk\s+selanjutnya\s+disebut|beralamat)", text_no_header, re.IGNORECASE)
    if match_penggugat_block:
        block = match_penggugat_block.group(1)
        # Ambil baris pertama sebagai nama, atau hingga ", pekerjaan" / ", umur"
        name_match = re.match(r"^\s*([\w\s.,()'-]+?)(?:,|\s*;?\s*\n\s*(?:umur|jenis kelamin|pekerjaan|agama|beralamat))", block.strip(), re.IGNORECASE)
        if name_match:
            penggugat_str = name_match.group(1).strip().replace('\n', ' ')
        else: # Fallback: ambil beberapa kata pertama dari blok
            penggugat_str = " ".join(block.strip().split()[:5])


    # Pola dasar untuk Tergugat/Termohon
    tergugat_str = "N/A"
    match_tergugat_block = re.search(r"(?:melawan|terhadap)\s*[:\n]([\s\S]+?)(?:;\s*\n?\s*yang\s+dalam\s+hal\s+ini|;\s*dan\s*untuk\s+selanjutnya\s+disebut|Pengadilan Negeri tersebut)", text_no_header, re.IGNORECASE)
    if match_tergugat_block:
        block = match_tergugat_block.group(1)
        name_match = re.match(r"^\s*([\w\s.,()'-]+?)(?:,|\s*;?\s*\n\s*(?:umur|jenis kelamin|pekerjaan|agama|beralamat|dahulu beralamat))", block.strip(), re.IGNORECASE)
        if name_match:
            tergugat_str = name_match.group(1).strip().replace('\n', ' ')
        else: # Fallback
             tergugat_str = " ".join(block.strip().split()[:5])

    # Bersihkan titik koma di akhir nama jika ada
    penggugat_str = re.sub(r";\s*$", "", penggugat_str).strip()
    tergugat_str = re.sub(r";\s*$", "", tergugat_str).strip()

    if penggugat_str != "N/A" and tergugat_str != "N/A":
      return f"Penggugat/Pemohon: {penggugat_str} VS Tergugat/Termohon: {tergugat_str}"
    elif penggugat_str != "N/A":
      return f"Penggugat/Pemohon: {penggugat_str}"
    elif tergugat_str != "N/A": # Jarang terjadi tanpa Penggugat
      return f"Tergugat/Termohon: {tergugat_str}"
    return "Pihak tidak teridentifikasi dari teks"


def extract_pasal_rules_from_text(text):
    """Mengekstrak pasal hukum dari teks."""
    # Pola lebih komprehensif, termasuk KUHPerdata, UU Perlindungan Anak, dll.
    pasal_pattern = re.compile(
        r"Pasal\s+[\w\d().-]+(?:.*?Ayat\s*\(?\s*\d+\s*\)?(?:.*?(?:huruf|angka)\s+[\w\d().-]+)?)?\s+"
        r"(?:Kitab\s+Undang-Undang\s+Hukum\s+(?:Pidana|Perdata|Acara\s+Pidana|Acara\s+Perdata)|KUH\s*Pidana|KUHPer(?:data)?|KUHAP|UU|Undang-Undang|Peraturan\s+Pemerintah|PP|Perma|PERMA)"
        r"(?:\s+Nomor\s+[\w\d/-]+\s+Tahun\s+\d{4})?",
        re.IGNORECASE
    )
    matches = pasal_pattern.findall(text)
    # Bersihkan dan unikkan
    cleaned_matches = sorted(list(set([re.sub(r'\s+', ' ', m).strip("., \n:") for m in matches])))
    return cleaned_matches if cleaned_matches else []

def extract_facts_summary(text):
    """Ekstrak Ringkasan Fakta / Duduk Perkara."""
    try:
        # Mencari awal bagian "DUDUK PERKARA" atau variasinya
        start_keywords = r"(?:TENTANG\s+DUDUK\s+PERKARA(?:NYA)?|DUDUK\s+PERKARA)"
        start_match = re.search(start_keywords, text, re.IGNORECASE)
        if not start_match: return ""
        start_index = start_match.end()

        # Mencari akhir bagian, sebelum "PERTIMBANGAN HUKUM" atau petitum Penggugat
        # atau sebelum "bukti-bukti" jika itu menandai akhir dari narasi fakta penggugat
        end_keywords = r"(?:TENTANG\s+PERTIMBANGAN\s+HUKUM|Menimbang,\s+bahwa\s+untuk\s+menguatkan\s+dalil|Menimbang,\s+bahwa\s+Penggugat\s+telah\s+mengajukan\s+bukti|Berdasarkan\s+alasan-?\s*alasan\s+tersebut\s+di\s*atas,\s*Penggugat\s+mohon|Maka\s+berdasarkan\s+hal-hal\s+tersebut\s+di\s*atas)"
        end_match = re.search(end_keywords, text[start_index:], re.IGNORECASE)

        fakta_text = ""
        if end_match:
            fakta_text = text[start_index : start_index + end_match.start()]
        else: # Fallback jika tidak ada penanda akhir yang jelas
            fakta_text = text[start_index : start_index + 5000] # Ambil blok teks
            last_double_newline = fakta_text.rfind('\n\n')
            if last_double_newline != -1:
                fakta_text = fakta_text[:last_double_newline]
        return fakta_text.strip()
    except Exception: return ""

def extract_legal_arguments(text):
    """Ekstrak Argumen Hukum / Pertimbangan Hukum."""
    try:
        # Mencari awal "PERTIMBANGAN HUKUM" atau blok "Menimbang, bahwa..." yang signifikan
        start_keywords = r"(?:TENTANG\s+PERTIMBANGAN\s+HUKUM|PERTIMBANGAN\s+HUKUM\s*:|Menimbang,\s+bahwa\s+setelah\s+Majelis\s+Hakim\s+memperhatikan)"
        start_match = re.search(start_keywords, text, re.IGNORECASE)

        start_index = 0
        if start_match:
            start_index = start_match.end()
        else: # Fallback jika header utama tidak ada, cari blok "Menimbang" pertama setelah fakta
            fakta_end_idx = 0
            temp_fakta = extract_facts_summary(text)
            if temp_fakta:
                fakta_end_idx = text.find(temp_fakta) + len(temp_fakta) if text.find(temp_fakta) != -1 else 0

            menimbang_match = re.search(r"Menimbang,\s+bahwa", text[fakta_end_idx:], re.IGNORECASE)
            if menimbang_match:
                start_index = fakta_end_idx + menimbang_match.start()
            else: # Jika semua gagal
                return ""

        # Mencari akhir sebelum "MENGADILI" atau "MEMUTUSKAN"
        end_keywords = r"(?:MENGADILI|MEMUTUSKAN)"
        end_match = re.search(end_keywords, text[start_index:], re.IGNORECASE)

        if end_match:
            return text[start_index : start_index + end_match.start()].strip()
        else: # Fallback: ambil sisa teks (mungkin terlalu banyak)
            return text[start_index:].strip()
    except Exception: return ""

print("\nMelakukan ekstraksi konten kunci dan feature engineering...")
# Terapkan fungsi ekstraksi
df_metadata['pihak_ekstraksi'] = df_metadata.apply(lambda row: extract_pihak_from_text(row['text_full_cleaned'], str(row['nomor_perkara'])), axis=1)
df_metadata['pasal_ekstraksi_teks'] = df_metadata['text_full_cleaned'].apply(extract_pasal_rules_from_text)
df_metadata['ringkasan_fakta_ekstraksi'] = df_metadata['text_full_cleaned'].apply(extract_facts_summary)
df_metadata['argumen_hukum_ekstraksi'] = df_metadata['text_full_cleaned'].apply(extract_legal_arguments)


Melakukan ekstraksi konten kunci dan feature engineering...


<ipython-input-6-f3d13afba1da>:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_metadata['pihak_ekstraksi'] = df_metadata.apply(lambda row: extract_pihak_from_text(row['text_full_cleaned'], str(row['nomor_perkara'])), axis=1)


In [ ]:
# --- 4. Feature Engineering Tambahan ---
import nltk
from nltk.tokenize import word_tokenize # Ensure this import is present
# Pastikan 'punkt' sudah diunduh untuk word_tokenize
try:
    nltk.data.find('tokenizers/punkt')
except LookupError: # <<< PERUBAHAN KUNCI DI SINI
    print("Mengunduh package 'punkt' untuk NLTK...")
    nltk.download('punkt')
    print("'punkt' berhasil diunduh.")

# Pastikan 'punkt' sudah diunduh untuk word_tokenize
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError: # <<< PERUBAHAN KUNCI DI SINI
    print("Mengunduh package 'punkt' untuk NLTK...")
    nltk.download('punkt_tab')
    print("'punkt' berhasil diunduh.")

def count_words_nltk(text_data):
    if pd.isna(text_data) or not isinstance(text_data, str) or text_data.strip() == "":
        return 0
    return len(word_tokenize(text_data))

df_metadata['jumlah_kata_teks'] = df_metadata['text_full_cleaned'].apply(count_words_nltk)

Mengunduh package 'punkt' untuk NLTK...
'punkt' berhasil diunduh.


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
# --- 5. Persiapan DataFrame Final ---
# Menggunakan 'case_id' dari CSV Tahap 1, ekstrak nomornya jika perlu.
# Format di CSV Tahap 1: f"case_{unique_id_counter:03d}"
try:
    df_metadata['id_kasus_numerik'] = df_metadata['case_id'].apply(lambda x: int(re.search(r'\d+', str(x)).group()) if pd.notna(x) and re.search(r'\d+', str(x)) else -1)
except Exception as e:
    print(f"Error saat parsing 'case_id' menjadi numerik: {e}. Menggunakan index sebagai fallback.")
    df_metadata['id_kasus_numerik'] = df_metadata.index + 1


# Pilih dan susun ulang kolom sesuai kebutuhan proyek
df_final_cases = pd.DataFrame({
    'case_id': df_metadata['id_kasus_numerik'],
    'no_perkara': df_metadata['nomor_perkara'],
    'tanggal': pd.to_datetime(df_metadata['tanggal_putusan'], errors='coerce').dt.strftime('%Y-%m-%d'), # Format tanggal
    'jenis_perkara': df_metadata['jenis_perkara'],
    'pihak': df_metadata['pihak_ekstraksi'], # Hasil ekstraksi dari teks
    'ringkasan_fakta': df_metadata['ringkasan_fakta_ekstraksi'],
    'pasal_diterapkan': df_metadata['pasal_ekstraksi_teks'], # Hasil ekstraksi dari teks (list)
    'argumen_hukum': df_metadata['argumen_hukum_ekstraksi'],
    'amar_putusan': df_metadata['amar_putusan'], # Dari CSV Tahap 1
    'jumlah_kata': df_metadata['jumlah_kata_teks'],
    'text_full': df_metadata['text_full_cleaned'],
    # Tambahkan kolom lain yang relevan dari df_metadata jika ada
    'link_sumber': df_metadata.get('link_sumber', pd.NA), # .get untuk handle jika kolom tdk ada
    'tahun_dokumen': df_metadata.get('tahun_dokumen', pd.NA)
})

# Bersihkan nilai NaN menjadi string kosong untuk kolom teks agar konsisten
cols_to_fillna_empty_str = ['no_perkara', 'tanggal', 'jenis_perkara', 'pihak', 'ringkasan_fakta', 'argumen_hukum', 'amar_putusan', 'text_full']
for col in cols_to_fillna_empty_str:
    if col in df_final_cases.columns:
        df_final_cases[col] = df_final_cases[col].fillna("")

# Kolom 'pasal_diterapkan' adalah list, biarkan apa adanya (atau konversi ke string jika perlu untuk beberapa format CSV)
df_final_cases['pasal_diterapkan'] = df_final_cases['pasal_diterapkan'].fillna("").apply(lambda x: x if isinstance(x, list) else [])


print("\nDataFrame final untuk Case Representation:")
print(df_final_cases.head())
print(f"\nKolom di DataFrame final: {df_final_cases.columns.tolist()}")
print(f"Jumlah kasus di DataFrame final: {len(df_final_cases)}")


DataFrame final untuk Case Representation:
   case_id             no_perkara     tanggal  \
0        1   28/Pdt.G/2020/PN Sdw               
1        2  229/Pdt.G/2023/PN Nga               
2        3   40/Pdt.G/2019/PN Ckr  2019-04-25   
3        4    9/Pdt.G/2025/PA.Sdk               
5        7   27/Pdt.G/2023/PN Nga               

                            jenis_perkara  \
0        Perdata Perdata Agama Perceraian   
1        Perdata Perdata Agama Perceraian   
2        Perdata Perdata Agama Perceraian   
3  Perdata Agama Perdata Agama Perceraian   
5        Perdata Perdata Agama Perceraian   

                                   pihak  \
0  Pihak tidak teridentifikasi dari teks   
1  Pihak tidak teridentifikasi dari teks   
2  Penggugat/Pemohon: MERY YANTI L. GAOL   
3             Penggugat/Pemohon: PEMOHON   
5  Pihak tidak teridentifikasi dari teks   

                                     ringkasan_fakta  \
0  Menimbang, bahwa Penggugat dengan surat gugata...   
1  Menimbang,

<ipython-input-9-7458814fddda>:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  'tanggal': pd.to_datetime(df_metadata['tanggal_putusan'], errors='coerce').dt.strftime('%Y-%m-%d'), # Format tanggal


In [ ]:
# --- 6. Penyimpanan Hasil Tahap 2 ---
# Sesuai PDF: /data/processed/cases.csv atau .json
OUTPUT_CSV_TAHAP2 = os.path.join(PATH_PROCESSED_DATA_OUTPUT, "cases_represented.csv")
OUTPUT_JSON_TAHAP2 = os.path.join(PATH_PROCESSED_DATA_OUTPUT, "cases_represented.json")

try:
    # Saat menyimpan ke CSV, kolom list (seperti 'pasal_diterapkan') mungkin perlu perlakuan khusus
    # agar bisa dibaca kembali dengan benar (misalnya, join list menjadi string).
    # Untuk kemudahan, kita bisa biarkan Pandas menyimpannya apa adanya,
    # dan saat memuat kembali, gunakan converter jika perlu.
    df_final_cases.to_csv(OUTPUT_CSV_TAHAP2, index=False, encoding='utf-8-sig')
    print(f"\nRepresentasi kasus berhasil disimpan ke CSV: {OUTPUT_CSV_TAHAP2}")
except Exception as e:
    print(f"\nError saat menyimpan CSV Tahap 2: {e}")

try:
    # Untuk JSON, format 'records' dengan lines=True baik untuk dibaca per baris.
    df_final_cases.to_json(OUTPUT_JSON_TAHAP2, orient='records', lines=True, indent=4, force_ascii=False)
    print(f"Representasi kasus berhasil disimpan ke JSON: {OUTPUT_JSON_TAHAP2}")
except Exception as e:
    print(f"\nError saat menyimpan JSON Tahap 2: {e}")


Representasi kasus berhasil disimpan ke CSV: /content/drive/MyDrive/Semester 6/PK/UAS/data/processed/cases_represented.csv
Representasi kasus berhasil disimpan ke JSON: /content/drive/MyDrive/Semester 6/PK/UAS/data/processed/cases_represented.json
